# AG2
AG2 (formerly AutoGen) is an open-source programming framework for building AI agents and facilitating their cooperation to solve tasks. AG2 supports tool use, autonomous and human-in-the-loop workflows, and multi-agent conversation patterns.

## Quick Start
### Chat with an Agent

In [4]:
# 1. Import our agent class
from autogen import ConversableAgent, config_list_from_json

# 2. Define our LLM configuration for OpenAI's GPT-4o mini
#    uses the OPENAI_API_KEY environment variable
# llm_config = {"api_type": "openai", "model": "gpt-4o-mini"}
llm_config = {
    "config_list": config_list_from_json(env_or_file="OAI_CONFIG_LIST")
}

# 3. Create our LLM agent
my_agent = ConversableAgent(
    name="helpful_agent",
    llm_config=llm_config,
    system_message="You are a poetic AI assistant, respond in rhyme.",
)

# 4. Run the agent with a prompt, type `exit` to end the conversation
chat_result = my_agent.run("In one sentence, what's the big deal about AI?")

# 5. Print the chat
print(chat_result.chat_history)

user (to helpful_agent):

In one sentence, what's the big deal about AI?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
helpful_agent (to user):

AI can learn and think, transforming how we create, innovate, and connect, shaping our future at an astonishing rate.

--------------------------------------------------------------------------------
[{'content': "In one sentence, what's the big deal about AI?", 'role': 'assistant', 'name': 'user'}, {'content': 'AI can learn and think, transforming how we create, innovate, and connect, shaping our future at an astonishing rate.', 'role': 'user', 'name': 'helpful_agent'}]


In [5]:
chat_result

ChatResult(chat_id=None, chat_history=[{'content': "In one sentence, what's the big deal about AI?", 'role': 'assistant', 'name': 'user'}, {'content': 'AI can learn and think, transforming how we create, innovate, and connect, shaping our future at an astonishing rate.', 'role': 'user', 'name': 'helpful_agent'}], summary='AI can learn and think, transforming how we create, innovate, and connect, shaping our future at an astonishing rate.', cost={'usage_including_cached_inference': {'total_cost': 2.01e-05, 'gpt-4o-mini-2024-07-18': {'cost': 2.01e-05, 'prompt_tokens': 34, 'completion_tokens': 25, 'total_tokens': 59}}, 'usage_excluding_cached_inference': {'total_cost': 0}}, human_input=['exit'])

## Two Agents Chat

In [6]:
# Chat between two comedian agents

# 1. Import our agent class
from autogen import ConversableAgent, config_list_from_json

# 2. Define our LLM configuration for OpenAI's GPT-4o mini,
#    uses the OPENAI_API_KEY environment variable
# llm_config = {"api_type": "openai", "model": "gpt-4o-mini"}
llm_config = {
    "config_list": config_list_from_json(env_or_file="OAI_CONFIG_LIST")
}

# 3. Create our agents who will tell each other jokes,
#    with Jack ending the chat when Emma says FINISH
jack = ConversableAgent(
    "Jack",
    llm_config=llm_config,
    system_message=(
      "Your name is Jack and you are a comedian "
      "in a two-person comedy show."
    ),
    is_termination_msg=lambda x: True if "FINISH" in x["content"] else False
)
emma = ConversableAgent(
    "Emma",
    llm_config=llm_config,
    system_message=(
      "Your name is Emma and you are a comedian "
      "in a two-person comedy show. Say the word FINISH "
      "ONLY AFTER you've heard 2 of Jack's jokes."
    ),
)

# 4. Run the chat
chat_result = jack.initiate_chat(
    emma,
    message="Emma, tell me a joke about goldfish and peanut butter.",
)

# 5. Print the chat
print(chat_result.chat_history)

Jack (to Emma):

Emma, tell me a joke about goldfish and peanut butter.

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Emma (to Jack):

Jack, why did the goldfish bring peanut butter to the party? Because it heard the snacks were going to be "fin-tastic!"

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Jack (to Emma):

That's a good one, Emma! You know, I always thought goldfish were kind of like introverts—they hang out in their bowl, minding their own business, and then suddenly, they're expected to entertain all these "snack" guests. I guess that’s why they bring peanut butter—they're just trying to spread some joy!

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
Emma (to Jack):

Haha, that's clever, Jack! You know, I tried to teach my goldfish some tricks once. I thought it could learn to fet

## Group Chat

In [7]:
# Group chat amongst agents to create a 4th grade lesson plan
# Flow determined by Group Chat Manager automatically, and
# should be Teacher > Planner > Reviewer > Teacher (repeats if necessary)

# 1. Import our agent and group chat classes
from autogen import ConversableAgent, GroupChat, GroupChatManager, config_list_from_json

# Define our LLM configuration for OpenAI's GPT-4o mini
# uses the OPENAI_API_KEY environment variable
# llm_config = {"api_type": "openai", "model": "gpt-4o-mini"}
llm_config = {
    "config_list": config_list_from_json(env_or_file="OAI_CONFIG_LIST")
}

# Planner agent setup
planner_message = "Create lesson plans for 4th grade. Use format: <title>, <learning_objectives>, <script>"
planner = ConversableAgent(
    name="planner_agent",
    llm_config=llm_config,
    system_message=planner_message,
    description="Creates lesson plans"
)

# Reviewer agent setup
reviewer_message = "Review lesson plans against 4th grade curriculum. Provide max 3 changes."
reviewer = ConversableAgent(
    name="reviewer_agent",
    llm_config=llm_config,
    system_message=reviewer_message,
    description="Reviews lesson plans"
)

# Teacher agent setup
teacher_message = "Choose topics and work with planner and reviewer. Say DONE! when finished."
teacher = ConversableAgent(
    name="teacher_agent",
    llm_config=llm_config,
    system_message=teacher_message,
)

# Setup group chat
groupchat = GroupChat(
    agents=[teacher, planner, reviewer],
    speaker_selection_method="auto",
    messages=[]
)

# Create manager
# At each turn, the manager will check if the message contains DONE! and end the chat if so
# Otherwise, it will select the next appropriate agent using its LLM
manager = GroupChatManager(
    name="group_manager",
    groupchat=groupchat,
    llm_config=llm_config,
    is_termination_msg=lambda x: "DONE!" in (x.get("content", "") or "").upper()
)

# Start the conversation
chat_result = teacher.initiate_chat(
    recipient=manager,
    message="Let's teach the kids about the solar system."
)

# Print the chat
print(chat_result.chat_history)

teacher_agent (to group_manager):

Let's teach the kids about the solar system.

--------------------------------------------------------------------------------

Next speaker: planner_agent


>>>>>>>> USING AUTO REPLY...
planner_agent (to group_manager):

**Title: Exploring the Solar System**

**Learning Objectives:**
1. Students will be able to identify and name the eight planets in our solar system.
2. Students will understand the difference between terrestrial and gas giant planets.
3. Students will be able to describe one unique feature of each planet.
4. Students will develop skills in teamwork and presentation.

**Script:**

**Introduction (10 minutes)**
- Teacher: "Good morning, class! Today, we’re going to take an exciting journey through our solar system. Can anyone tell me how many planets we have in our solar system?"
- [Allow students to respond.]
- Teacher: "That's right! There are eight planets. We are going to learn about each of them and discover some interesting facts